In [56]:
import boto3
import json
from typing import Dict, Any
from dotenv import load_dotenv
import os

In [57]:
load_dotenv()
region = os.environ.get('AWS_REGION')
MODEL_ID = os.environ.get('MODEL_ID')
role_arn = os.environ.get('ROLE_ARN')
knowledge_base_id = os.environ.get('KNOWLEDGE_BASE_ID')

In [58]:
bedrock_agent = boto3.client('bedrock-agent', region_name = region)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name = region)

In [ ]:
flow_name="Routers"
flow_description="A example of routing based on condition"

In [60]:
# The input node validates that the content of the InvokeFlow request is a JSON object.
input_node = {
    "type": "Input",
    "name": "FlowInput",
    "outputs": [
        {
            "name": "document",
            "type": "String"
        }
    ]
}

In [ ]:
intent_prompt = f"""Identify the intent from the text provided.  If the question is related to Agentic Memory, say "agents" if not say "others".  just provide
                    the intent.  do not add any preambles.  Question: {{query}}"""

intent_node = {
    "type": "Prompt",
    "name": "detectintent",
    "configuration": {
        "prompt": {
            "sourceConfiguration": {
                "inline": {
                    "modelId": MODEL_ID,
                    "templateType": "TEXT",
                    "inferenceConfiguration": {
                        "text": {
                            "temperature": 0.8
                        }
                    },
                    "templateConfiguration": { 
                        "text": {
                            "text": intent_prompt
                        }
                    }
                }
            }
        }
    },
    "inputs": [
        {
            "name": "query",
            "type": "String",
            "expression": "$.data"
        },
    ],
    "outputs": [
        {
            "name": "modelCompletion",
            "type": "String"
        }
    ]
}

In [ ]:
conditional_node = {
        "name": "ConditionalNode",
        "type": "Condition",
        "configuration": {
            "condition": {
                "conditions": [
                    {
                        "expression": "conditionInput == \"agents\"",
                        "name": "Condition"
                    },
                    {
                        "name": "default"
                    }
                ]
            }
        },
        "inputs": [
            {
                "name": "conditionInput",
                "type": "String",
                "expression": "$.data"

            }
        ],

    }

In [61]:
web_search_node = {
        "name": "web_search",
        "type": "LambdaFunction",
        "configuration" : {
            'lambdaFunction': {
                'lambdaArn': 'arn:aws:lambda:us-west-2:924155096146:function:web_search'
            },
        },
        "inputs": [
            {
                "name": "codeHookInput",
                "type": "String",
                "expression": "$.data"
            }
        ],
        "outputs": [
            {
                "name": "functionResponse",
                "type": "String"
            }
        ]
}

In [62]:
kb_search_node = {
        "name": "kb_search_node",
        "type": "KnowledgeBase",
        "configuration" : {
            'knowledgeBase': {
                'knowledgeBaseId': knowledge_base_id,
                'modelId': MODEL_ID
            }
        },
        "inputs": [
            {
                "name": "retrievalQuery",
                "type": "String",
                "expression": "$.data"
            }
        ],
        "outputs": [
            {
                "name": "outputText",
                "type": "String"
            }
        ]
        
}

In [ ]:
blogger_node = {
    "type": "Prompt",
    "name": "blogger",
    "configuration": {
        "prompt": {
            "sourceConfiguration": {
                "inline": {
                    "modelId": MODEL_ID,
                    "templateType": "TEXT",
                    "inferenceConfiguration": {
                        "text": {
                            "temperature": 0.8
                        }
                    },
                    "templateConfiguration": { 
                        "text": {
                            "text": "Create a succint summary from the context: {{webresults}} and {{kbresults}}."
                        }
                    }
                }
            }
        }
    },
    "inputs": [
        {
            "name": "webresults",
            "type": "String",
            "expression": "$.data"
        },
        {
            "name": "kbresults",
            "type": "String",
            "expression": "$.data"
        }
    ],
    "outputs": [
        {
            "name": "modelCompletion",
            "type": "String"
        }
    ]
}

In [64]:
output_node = {
    "type": "Output",
    "name": "FlowOutput",
    "inputs": [
        {
            "name": "document",
            "type": "String",
            "expression": "$.data"
        }
    ]
}

In [ ]:
connections =  [
    {
      "name": "Input_to_intent",
      "source": input_node['name'],
      "target": intent_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": intent_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "",
      "source": intent_node['name'],
      "target": condition_node['name'],
      "type": "Conditional",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": intent_node['inputs'][0]['name']
          }
      }
    }    

    {
      "name": "Input_to_webserach",
      "source": input_node['name'],
      "target": web_search_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": web_search_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "Input_to_KB",
      "source": input_node['name'],
      "target": kb_search_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": kb_search_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "websearch_to_blog",
      "source": web_search_node['name'],
      "target": blogger_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": web_search_node['outputs'][0]['name'], 
            "targetInput": blogger_node['inputs'][0]['name'],
          },
      }
    },
    {
      "name": "kbsearch_to_blog",
      "source": kb_search_node['name'],
      "target": blogger_node['name'],
      "type": "Data",
      "configuration": {
          "data": {
            "sourceOutput": kb_search_node['outputs'][0]['name'], 
            "targetInput": blogger_node['inputs'][1]['name']
          }
      }
    },
    {
      "name": "blog_to_output",
      "source": blogger_node['name'],
      "target": output_node['name'],
      "type": "Data",
      "configuration": {
        "data": {
          "sourceOutput": blogger_node['outputs'][0]['name'],
          "targetInput": output_node['inputs'][0]['name']
        }
      }
    }
]

In [66]:
response = bedrock_agent.create_flow(
    name=flow_name,
    description=flow_description,
    executionRoleArn=role_arn,
    definition={
        "nodes": [input_node, web_search_node, kb_search_node, blogger_node, output_node],
        "connections": connections
    }
)

In [67]:
flow_id = response.get("id")

In [68]:
bedrock_agent.prepare_flow(flowIdentifier=flow_id)

{'ResponseMetadata': {'RequestId': 'c36d7b23-6e52-44c0-993c-ceb63acb19b1',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 31 Mar 2025 14:52:36 GMT',
   'content-type': 'application/json',
   'content-length': '40',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c36d7b23-6e52-44c0-993c-ceb63acb19b1',
   'x-amz-apigw-id': 'IS_YKGBKPHcEDsg=',
   'x-amzn-trace-id': 'Root=1-67eaac33-25b4c4df1878352345e77edd'},
  'RetryAttempts': 0},
 'id': 'AIIM4VQXT8',
 'status': 'Preparing'}

In [69]:
response = bedrock_agent.create_flow_version(flowIdentifier=flow_id)
                                
flow_version = response.get("version")

In [70]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier=flow_id,
    name="latest",
    description="Alias pointing to the latest version of the flow.",
    routingConfiguration=[
        {
            "flowVersion": flow_version
        }
    ]
)

flow_alias_id = response.get("id")

In [71]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier=flow_id,
    flowAliasIdentifier=flow_alias_id,
    inputs=[
        {
            "content": {
                "document": "agentic memory"
            },
            "nodeName": "FlowInput",
            "nodeOutputName": "document"
        }
    ]
)

result = {}

for event in response.get("responseStream"):
    result.update(event)

if result['flowCompletionEvent']['completionReason'] == 'SUCCESS':
    print("Flow invocation was successful! The output of the flow is as follows:\n")
    print(result['flowOutputEvent']['content']['document'])

else:
    print("The flow invocation completed because of the following reason:", result['flowCompletionEvent']['completionReason'])

Flow invocation was successful! The output of the flow is as follows:

Here is a succinct summary:

The paper proposes A-MEM, a novel agentic memory system for large language model (LLM) agents. A-MEM allows LLM agents to dynamically organize memories in an agentic way and enables memory evolution, where new memories can trigger updates to existing historical memories. It combines structured organization principles from Zettelkasten with flexible agent-driven decision making for adaptive, context-aware memory management. The key memory modules include semantic memory for world knowledge, episodic memory for case-related content and past decision processes, and procedural memory containing the LLM's implicit knowledge and the agent's coded knowledge.
